프리미어리그 2016-2022시즌 득점 순위, 필드골 에서 슈팅 당 골 비율

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54'}


name = []
team = []
score = []
assist = []
shoot = []
penalty_kick = []

for i in range(2016,2022):
    
    url =f"https://sports.news.naver.com/wfootball/record/index?category=epl&year={i}&tab=player"

    r=requests.get(url, headers = headers)
    r.status_code

    soup =  BeautifulSoup(r.text, 'html.parser')
    
    player_info = []
    player_info = soup.find_all('tr')
    del player_info[0]
    

    #player_info에서 원하는 태그만 모두 찾기
    for info in player_info:
        player = info.find_all("strong")
        #print(player)
        #print(player[0])
        player1 = info.find_all("span")
        #print(player1[11])

        #해당 정보를 찾아서 각 리스트에 append로 추가
        
        name.append(player1[1].text)
        team.append(player1[2].text)
        score.append(player1[3].text)
        assist.append(player1[4].text)
        shoot.append(player1[6].text)
        penalty_kick.append(player1[11].text)
    
    #1페이지 크롤링하고 1초 쉬기
    time.sleep(1)
    
    #2021/2022 pd.DataFrame() 로 저장하기
    df = pd.DataFrame(
        {
         "name" : name,
         "team" : team,
         "score" : score,
         "assist" : assist,
         "shoot" : shoot,
         "penalty_kick" : penalty_kick}
    )
    
#astype으로 score 타입 변환
df = df.astype({'team':'str', 'score':'int' , 'assist':'int' , 'shoot':'int', 'penalty_kick':'int'})

#name 묶고 인뎃스 초기화
df2016_2022 = df.groupby('name').sum()
df2016_2022 = df2016_2022.reset_index()

#goals_scored 많은 순으로 정렬
players2016_2022data_top_score= df2016_2022.sort_values(by='score' , ascending=False)

#득점 top 10명 만 시각화
players2016_2022data_goals_scored_top10 = players2016_2022data_top_score.head(10)

colors = ['#51F74F',] * len(players2016_2022data_goals_scored_top10['name'])
colors[0] = '#57CBEB'

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(

    go.Bar(
        x=players2016_2022data_goals_scored_top10['name'],
        y=players2016_2022data_goals_scored_top10['score'],
        text=players2016_2022data_goals_scored_top10['score'],
        textposition='auto',
        texttemplate='%{text:}골',
        marker_color=colors
    )
) 
fig.update_layout(
    {
        "title": {
            "text": "<b>프리미어리그 16-22시즌 득점왕</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "선수이름",
            "showticklabels":True,
            "tickfont": {
                "size": 12
            }
        },
        "yaxis": {
            "title": "득점수"
        },
        "template":'plotly_white'
    }
)

fig.add_annotation(
            x="해리 케인", #x축 선정값
            y=134, #y축 선정값
            text="<b>EPL[16-22]시즌 TOP 득점</b>", #텍스트 문구 설정
            showarrow=True,
            font=dict(
                size=13,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#F25D50",
            ax=20,
            ay=-30,
            bordercolor="#F25D50",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F25D50",
            opacity=0.9 #투명도
)
fig.show()

#shoot_goal_rate 컬럼 만들기
players2016_2022data_top_score["shoot_goal_rate"] = players2016_2022data_top_score['shoot'] / (players2016_2022data_top_score["score"] - players2016_2022data_top_score["penalty_kick"])

#inf 값 NaN으로 replace
players2016_2022data_top_score = players2016_2022data_top_score.replace([np.inf, -np.inf], np.nan)

#결측치(NaN) 값 0으로 변환 후 minutes_goal_rate컬럼 astype로 데이터 타입 변경
players2016_2022data_top_score= players2016_2022data_top_score.fillna(0)
players2016_2022data_top_score = players2016_2022data_top_score.astype({'shoot_goal_rate':'float'})

#어느 정도 득점이 있는 선수들 중 뽑아내야하기에 40득점 이상인 선수만 가져오기
players2016_2012shoot_goal_rate = players2016_2022data_top_score[players2016_2022data_top_score['score']>30]

#shoot_goal_rate 값 정렬
players2016_2012shoot_goal_rate = players2016_2012shoot_goal_rate.sort_values(by='shoot_goal_rate' , ascending=True)
#솟숫점 두번쨰 자리에서 반올림
players2016_2012shoot_goal_rate = players2016_2012shoot_goal_rate.round(3)

#players2016_2012shoot_goal_rate top 10명 만 시각화
players2016_2012shoot_goal_rate_top10 = players2016_2012shoot_goal_rate.head(10)

colors = ['#51F74F',] * len(players2016_2012shoot_goal_rate_top10['name'])
colors[0] = '#57CBEB'

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(

    go.Bar(
        x=players2016_2012shoot_goal_rate_top10['name'],
        y=players2016_2012shoot_goal_rate_top10['shoot_goal_rate'],
        text=players2016_2012shoot_goal_rate_top10['shoot_goal_rate'],
        textposition='auto',
        texttemplate='%{text:}개 슈팅 당 한골',
        marker_color=colors
    )
) 
fig.update_layout(
    {
        "title": {
            "text": "<b>프리미어리그 16-22시즌 골 결정력</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "선수이름",
            "showticklabels":True,
            "tickfont": {
                "size": 12
            }
        },
        "yaxis": {
            "title": "슈팅당 필드골 비율"
        },
        "template":'plotly_white'
    }
) 

fig.add_annotation(
            x="로멜루 루카쿠", #x축 선정값
            y=4.865, #y축 선정값
            text="<b>EPL[16-22]시즌 TOP골 결정력</b>", #텍스트 문구 설정
            showarrow=True,
            font=dict(
                size=13,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#F25D50",
            ax=20,
            ay=-30,
            bordercolor="#F25D50",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F25D50",
            opacity=0.9 #투명도
)
fig.show()

트랜스퍼마켓 축구선수 포지션별 가치 확인해보기 https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0

In [9]:
#트랜스퍼마켓 필요 데이터 크롤링 1페이지 부터 25페이지 까지 총 250개

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54'}


#7개 정보를 담을 빈 리스트 만들기 number, name, postion, age, nation, team, value
number = []
name = []
postion = []
age = []
nation = []
team = []
value = []

for i in range(1,21):

    url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={i}"

    r=requests.get(url, headers = headers)
    r.status_code

    soup =  BeautifulSoup(r.text, 'html.parser')
    #print(soup)

    #선수들의 정보가 담긴 태그와 클래스 찾기
    player_info = soup.find_all('tr',class_= ['odd','even'])
    #player_info 첫번쨰 것만 확인
    #print(player_info[0])
    #전체 개수 확인하기
    #print(len(player_info))

   

    #player_info에서 'td' 태그만 모두 찾기
    for info in player_info:
        player = info.find_all("td")
        #print(player)
        #print(player[0])
    #해당 정보를 찾아서 각 리스트에 append로 추가
        number.append(player[0].text)
        name.append(player[3].text)
        postion.append(player[4].text)
        age.append(player[5].text)
        nation.append(player[6].img['alt'])
        team.append(player[7].img['alt'])
        value.append(player[8].text.strip())
    
    
    
    
    #pd.DataFrame() 로 저장하기
    df = pd.DataFrame(
        {"number": number,
         "name" : name,
         "postion" : postion,
         "age" : age,
         "nation" : nation,
         "team" : team,
         "value" : value}
    )
    

#필요한 데이터만 확인
players_postion_value  = df[['postion','value']]
#value컬럼 숫자만 추출하기
players_postion_value['value'] = players_postion_value['value'].str.extract(r'(\d+)')


#동일 포지션 횟수 확인(나중에 SUM한 value에서 나눠줘야 하기 떄문)
ps = pd.DataFrame(players_postion_value["postion"].value_counts(ascending=True))
#리셋 인덱스
ps = ps.reset_index()
#컬럼 명 재설정
ps.rename(columns={'postion' : 'counts', 'index' : 'postion'}, inplace=True)

#value 타입 변경
players_postion_value = players_postion_value.astype({'value':'int'})

#포지션별 groupby
players_postion_value = players_postion_value.groupby('postion').sum()
#리셋 인덱스
players_postion_value = players_postion_value.reset_index()

#아까 만든 ps 데이터 프레임 과 merge
merger_ps_players_postion_value = pd.merge(ps, players_postion_value, on='postion')

#포지션별 평균가치 컬럼 만들기
merger_ps_players_postion_value["real_value"] = merger_ps_players_postion_value["value"] / merger_ps_players_postion_value["counts"]

#minutes_goal_rate 값 정렬 하고 소숫점 두자리 까지 표현
merger_ps_players_postion_value= merger_ps_players_postion_value.sort_values(by='real_value' , ascending=False)
merger_ps_players_postion_value = merger_ps_players_postion_value.round(2)

#포지션별 가치 순위
colors = ['#51F74F',] * len(merger_ps_players_postion_value['real_value'])
colors[0] = '#57CBEB'

#시각화
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(

    go.Bar(
        x=merger_ps_players_postion_value['postion'],
        y=merger_ps_players_postion_value['real_value'],
        text=merger_ps_players_postion_value['real_value'],
        textposition='auto',
        texttemplate='%{text:}€',
        marker_color=colors
    )
) 
fig.update_layout(
    {
        "title": {
            "text": "<b>트랜스퍼마켓 포지션별 가치</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "포지션 명",
            "showticklabels":True,
            "tickfont": {
                "size": 12
            }
        },
        "yaxis": {
            "title": "유로"
        },
        "template":'plotly_white'
    }
) 

fig.add_annotation(
            x="Left Winger", #x축 선정값
            y=38.45, #y축 선정값
            text="<b>TOP expensive in position</b>", #텍스트 문구 설정
            showarrow=True,
            font=dict(
                size=13,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#F25D50",
            ax=20,
            ay=-30,
            bordercolor="#F25D50",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F25D50",
            opacity=0.9 #투명도
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7664\3301567522.py:72: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



맨시티, 리버풀, 첼시 데이터 비교

In [44]:
#공격 지표 확인
import pandas as pd
import numpy as np

england_premier_league_squad_defensive_actions_22=pd.read_excel("england_premier_league_squad_defensive_actions_22.xls")
england_premier_league_squad_goal_shot_creation_22=pd.read_excel("england_premier_league_squad_goal_shot_creation_22.xls")
england_premier_league_squad_passing_stats_22=pd.read_excel("england_premier_league_squad_passing_stats_22.xls")
england_premier_league_squad_possession_22=pd.read_excel("england_premier_league_squad_possession_22.xls")
england_premier_league_squad_shooting_22=pd.read_excel("england_premier_league_squad_shooting_22.xls")
england_premier_league_stats_22=pd.read_excel("england_premier_league_stats_22.xls")
england_premier_league_table_22=pd.read_excel("england_premier_league_table_22.xls")

#순위 팀명 컬럼만
england_premier_league_table_22 = england_premier_league_table_22[['Rk', 'Squad']]

#merge 해서 탑3개만 확인(맨시티, 리버풀, 첼시 데이터만 확인)
merger_table_shot = pd.merge(england_premier_league_table_22, england_premier_league_squad_shooting_22, on='Squad')
merger_table_shot = merger_table_shot.head(3)

merger_table_shot1 = pd.merge(england_premier_league_table_22, england_premier_league_squad_possession_22, on='Squad')
merger_table_shot1 = merger_table_shot1.head(3)

import plotly.offline
import plotly.express as px

fig = px.pie(merger_table_shot, values='G;SoT', names='Squad', title = '유효슛 당 골', color='Squad',
            color_discrete_map={'Manchester City':'#57CBEB',
                                 'Liverpool':'#D60000',
                                 'Chelsea':'#001FD6'
                                 })
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

fig = px.pie(merger_table_shot1, values='CPA', names='Squad', title = '18야드 박스로 운반', color='Squad',
            color_discrete_map={'Manchester City':'#57CBEB',
                                 'Liverpool':'#D60000',
                                 'Chelsea':'#001FD6'
                                 })
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [43]:
#패스 지표 확인
import pandas as pd
import numpy as np

england_premier_league_squad_defensive_actions_22=pd.read_excel("england_premier_league_squad_defensive_actions_22.xls")
england_premier_league_squad_goal_shot_creation_22=pd.read_excel("england_premier_league_squad_goal_shot_creation_22.xls")
england_premier_league_squad_passing_stats_22=pd.read_excel("england_premier_league_squad_passing_stats_22.xls")
england_premier_league_squad_possession_22=pd.read_excel("england_premier_league_squad_possession_22.xls")
england_premier_league_squad_shooting_22=pd.read_excel("england_premier_league_squad_shooting_22.xls")
england_premier_league_stats_22=pd.read_excel("england_premier_league_stats_22.xls")
england_premier_league_table_22=pd.read_excel("england_premier_league_table_22.xls")

#순위 팀명 컬럼만
england_premier_league_table_22 = england_premier_league_table_22[['Rk', 'Squad']]

#merge 해서 탑3개만 확인(맨시티, 리버풀, 첼시 데이터만 확인)
merger_table_shot3 = pd.merge(england_premier_league_table_22, england_premier_league_squad_passing_stats_22, on='Squad')
merger_table_shot3 = merger_table_shot3.head(3)

import plotly.offline
import plotly.express as px

fig = px.pie(merger_table_shot3, values='TotCmpRt', names='Squad', title = '패스 성공률', color='Squad',
            color_discrete_map={'Manchester City':'#57CBEB',
                                 'Liverpool':'#D60000',
                                 'Chelsea':'#001FD6'
                                 })
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

fig = px.pie(merger_table_shot3, values='ProgPs', names='Squad', title = '진보적인 패스', color='Squad',
            color_discrete_map={'Manchester City':'#57CBEB',
                                 'Liverpool':'#D60000',
                                 'Chelsea':'#001FD6'
                                 })
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [56]:
#압박 지표 확인
import pandas as pd
import numpy as np

england_premier_league_squad_defensive_actions_22=pd.read_excel("england_premier_league_squad_defensive_actions_22.xls")
england_premier_league_squad_goal_shot_creation_22=pd.read_excel("england_premier_league_squad_goal_shot_creation_22.xls")
england_premier_league_squad_passing_stats_22=pd.read_excel("england_premier_league_squad_passing_stats_22.xls")
england_premier_league_squad_possession_22=pd.read_excel("england_premier_league_squad_possession_22.xls")
england_premier_league_squad_shooting_22=pd.read_excel("england_premier_league_squad_shooting_22.xls")
england_premier_league_stats_22=pd.read_excel("england_premier_league_stats_22.xls")
england_premier_league_table_22=pd.read_excel("england_premier_league_table_22.xls")

#순위 팀명 컬럼만
england_premier_league_table_22 = england_premier_league_table_22[['Rk', 'Squad']]

#merge 해서 탑3개만 확인(맨시티, 리버풀, 첼시 데이터만 확인)
merger_table_shot4 = pd.merge(england_premier_league_table_22, england_premier_league_squad_defensive_actions_22, on='Squad')
merger_table_shot4 = merger_table_shot4.head(3)

import plotly.offline
import plotly.express as px

fig = px.pie(merger_table_shot4, values='PressTot', names='Squad', title = '총 압박', color='Squad',
            color_discrete_map={'Manchester City':'#57CBEB',
                                 'Liverpool':'#D60000',
                                 'Chelsea':'#001FD6'
                                 } )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

fig = px.pie(merger_table_shot4, values='SuccPress', names='Squad', title = '성공적인 압박', color='Squad',
            color_discrete_map={'Manchester City':'#57CBEB',
                                 'Liverpool':'#D60000',
                                 'Chelsea':'#001FD6'
                                 })
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()